<a href="https://colab.research.google.com/github/zinojeng/chart_review/blob/main/Chart_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from getpass import getpass
openai_api_key = getpass('請輸入你的 OpenAI API Key: ')

In [ ]:
import requests
import json
from pprint import pp

In [ ]:
def get_completion(messages, model="gpt-4", temperature=0, max_tokens=2000):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

In [ ]:
with open('/content/sample_data/726752.txt', 'r') as file:
    adm_note = file.read()

# **出院病歷品質審查表**

1.Chief complaint 包括Symptoms和Duration。
□2：符合 (無symptoms者需註明疾病診斷日期及來院目的)

□1：部分符合(未寫出duration)

□0：不符合 (只寫for C/T、處置或開刀)

2.Present illness呈現對病史、病徵及綜合歷程診治之有關重要資料。
□4 詳盡且分析主述病因具邏輯性

□2 描述部分相關症狀，但缺乏邏輯性

□0：待改善 (只簡列住院目的及數據)

3.Past/Personal/Family History、過敏史、社會關係史完整。
□2：符合   □1 (部分符合)   □0：不符合

4.器官系統評估完整，並對Positive finding有加註說明。
□3 (Positive finding有加註說明者)

 □2 (每一系統描述5項以上者)

 □1 (每一系統描述5項以下者)

□0：待改善(主訴及病史中有相關疑問，而ROS正常者)

5.Physical Examination中，需有記錄身高、體重、血壓、體溫、呼吸、心跳、Nutrition assessment及疼痛資料。
□1：符合
 □0：不符合

6.身體診查描述詳盡且合乎邏輯。
□3：內容符合病史與主訴且有強調positive finding (嬰幼兒
     例外)

□2：內容符合病史與主訴者

□1：每一系統描述3項以上者

□0：每一系統描述2項以下或不合邏輯性

7.專科身體診查是否書寫異常發現。
□1：符合

□0：不符合(如書寫Nil, As above)

8.Admission Note有完整初步診斷和診療計劃
□1：符合   
□0：不符合

9.Problem list頁中有依病情修正並呈現正確發病日期。
□2：符合

□0：不符合


# **Chain of Thought**
相互連接和基於彼此的思想或觀點，形成一個有進展的和有邏輯的思考過程

In [ ]:
system_promot = "You are a internal medical physician who specializes in writing and reviewing medical records. Let's think step by step and critically, Rrigorous and detailed review as possible as you can. \
give me the each and total score and each comment crtically more detail.\
If the writing is not good, incomplete, the content is short, and the context does not fit, please give low score or no score and Give feedback on why the score is so low, as detail as you can. \
response in zh-tw  "

#Few Shot starts here:

{"role":"user","content":"The patient has a medical history of atrial fibrillation. \
 The physical examination revealed a heart rate that was within the normal range and regular."},
{"role":"assistant","content":"The patient has an irregular heartbeat, and the heartbeat should be irregular, and this demerit points. \
The content of the present illness or system review does not correspond to the physical examination. \
For abnormal inspection results or "+", no relevant instructions are added."},

{"role":"user","content":"The chest pain began gradually and worsened when the individual was in a supine position. \
 The location was situated on the left anterior chest wall. "},
{"role":"assistant","content":"Graduation: Mode of onset is provided. \
left anterior chest wall: location provided. \
supine position: aggravation provided. \
No description of chest pain character, relief or related symptoms"},

user_prompt = f"""

According the content of admission note: {adm_note}

1. Chief complaint includes Symptoms and Duration
2 points will be awarded for those who meet the criteria (those without symptoms must indicate the date of disease diagnosis and the purpose of coming to the hospital)
1 point is awarded for partial compliance (duration is not written)
0 points if not met (write only for C/T, treatment or surgery)

2. Present illness presents important information on medical history, symptoms and comprehensive process diagnosis and treatment.
according to the content of admission note, proactively inquire using the LQQOPERA approach, which involves: \
1. Location 2. Character 3. Mode of onset 4. Precipitating , exaggerating and relieving factor 5. Associated symptoms \
indicate which item is mentioned and which item is not mentioned
Detailed and logical analysis of the cause of the disease, 4 points will be awarded
Describes some related symptoms, but lacks logic. Give 2 points.
To be improved (only briefly list hospitalization purpose and data), give 0 points

3. Complete Past/Personal/Family History, allergy history, and social relationship history
□Consistent: give 2 points
□ (Partially consistent): Give 1 point
□: Not consistent: give 0 points

4. The review system assessment is complete, and the Present illness description and Positive finding are annotated.
It is necessary to point out which item does not conform to the above content
(Positive finding has an explanation): give 3 points
□ (If each system describes more than 5 items): Give 2 points
□ (If each system describes less than 5 items): Give 1 point
□To be improved (those with relevant questions in the chief complaint and medical history, but ROS is normal): 0 points

5. In the Physical Examination, height, weight, blood pressure, body temperature, respiration, heartbeat, nutrition assessment and pain information must be recorded.
It is necessary to point out which item does not conform to the above content
□The content is consistent with the medical history and main complaints and emphasizes positive finding (infants and young children)
Exception): give 3 points
□The content is consistent with the medical history and main complainant: 2 points
□If each system describes more than 3 items: give 1 point
□Each system description contains less than 2 items or is illogical: give 0 points
6. Physical examination is described in detail and logically.
It is necessary to point out which item does not conform to the above content
□Consistent: Give 1 point
□Not conforming (such as writing Nil, As above): give 0 points

7. Specialist physical examination to see if any abnormalities in writing are found
□Consistent: Give 1 point
□Does not meet the requirement: give 0 points

8. Admission Note has a complete preliminary diagnosis and treatment plan
□Consistent: Give 1 point
□Does not meet the requirement: give 0 points

9. The Problem list page is corrected according to the condition and displays the correct onset date.
□Consistent: Give 1 point
□Does not meet the requirement: give 0 points
"""


messages = [
  { "role": "system", "content": system_promot },
  { "role": "user", "content": user_prompt }
]

result = get_completion(messages, temperature=0.0, model="gpt-4")
print(result)